In [1]:
import numpy as np
import numpy.linalg as la

In [2]:
datasets = ['CIFAR', 'MNIST']
net_names = ['ConvBig', 'ConvMed']
perturbations = ['Haze', 'ContrastVariation', 'Rotation']

In [3]:
import re

class Interval:
    def __init__(self, interval_str):
        m = re.match(r'\[(.*),(.*)\]', interval_str)
        self.lower, self.upper = float(m.group(1)), float(m.group(2))
    def get_abs_max(self):
        return max(abs(self.lower), abs(self.upper))

In [4]:
from statistics import median

NUM_IMAGES = 100

all_data = {}

for dataset in datasets:
    all_data[dataset] = {}
    for net in net_names:
        if dataset == 'MNIST' and net == 'ConvMed':
            continue
        
        all_data[dataset][net] = {}
        for perturbation in perturbations:
            all_data[dataset][net][perturbation] = {}
            filename = f'original/results/results_nosplit/{net}_{dataset}_{perturbation}_nosplit.txt'
            with open(filename) as f:
                content = f.readlines()
            content = [x.strip() for x in content] 
            data = []
            for header, arr in zip(content[::2], content[1::2]):
                items = header.split(',')
                interval_size = float(items[4])
                time = float(items[6])
                
                jacobians = np.array(list(map(lambda x: Interval(x).get_abs_max(), arr.split(';')[:-1]))).reshape(NUM_IMAGES, 10)
                
                avg_norm = 0
                for jacobi in jacobians:
                    avg_norm += la.norm(jacobi, np.inf)
                avg_norm /= NUM_IMAGES                
                data.append((interval_size, time, avg_norm))
            
            all_data[dataset][net][perturbation] = data

In [5]:
from statistics import median

NUM_IMAGES = 100

all_data_sound = {}

for dataset in datasets:
    all_data_sound[dataset] = {}
    for net in net_names:
        if dataset == 'MNIST' and net == 'ConvMed':
            continue
        
        all_data_sound[dataset][net] = {}
        for perturbation in perturbations:
            all_data_sound[dataset][net][perturbation] = {}
            filename = f'sound/results/results_nosplit/{net}_{dataset}_{perturbation}_nosplit.txt'
            with open(filename) as f:
                content = f.readlines()
            content = [x.strip() for x in content] 
            data = []
            for header, arr in zip(content[::2], content[1::2]):
                items = header.split(',')
                interval_size = float(items[4])
                time = float(items[6])
                
                jacobians = np.array(list(map(lambda x: Interval(x).get_abs_max(), arr.split(';')[:-1]))).reshape(NUM_IMAGES, 10)
                
                avg_norm = 0
                for jacobi in jacobians:
                    avg_norm += la.norm(jacobi, np.inf)
                avg_norm /= NUM_IMAGES                
                data.append((interval_size, time, avg_norm))
            
            all_data_sound[dataset][net][perturbation] = data

In [6]:
perturbations = ['HazeThenRotation', 'ContrastVariationThenRotation', 'ContrastVariationThenHaze']
interval_sizes = np.array([10**(-0.25*k) for k in range(4, 20, 3)]) * 2

from statistics import median

NUM_IMAGES = 10

for dataset in datasets:
    for net in net_names:
        if net == 'ConvMed' and dataset == 'MNIST':
            continue
            
        for perturbation in perturbations:
            filename = f'original/results_compose/results_compose_nosplit/{net}_{dataset}_{perturbation}_nosplit.txt'
            with open(filename) as f:
                content = f.readlines()
            content = [x.strip() for x in content] 
            data = []
            
            for header, arr in zip(content[::2], content[1::2]):
                items = header.split(',')
                interval_size = float(items[4])
                time = float(items[7])
                
                jacobians = np.array(list(map(lambda x: Interval(x).get_abs_max(), arr.split(';')[:-1]))).reshape(NUM_IMAGES, 20)
                
                avg_norm = 0
                for jacobi in jacobians:
                    jacobi = jacobi.reshape(2, 10).T
                    avg_norm += la.norm(jacobi, np.inf)
                avg_norm /= NUM_IMAGES
                
                add = 0
                for isize in interval_sizes:
                    if np.isclose(isize, interval_size):
                        add = 1
                        break
                if add:
                    data.append((interval_size, time, avg_norm))
                    
            all_data[dataset][net][perturbation] = data            

In [7]:
from statistics import median

NUM_IMAGES = 10

for dataset in datasets:
    for net in net_names:
        if net == 'ConvMed' and dataset == 'MNIST':
            continue
            
        for perturbation in perturbations:
            filename = f'sound/results_compose/results_compose_nosplit/{net}_{dataset}_{perturbation}_nosplit.txt'
            with open(filename) as f:
                content = f.readlines()
            content = [x.strip() for x in content] 
            data = []
            
            for header, arr in zip(content[::2], content[1::2]):
                items = header.split(',')
                interval_size = float(items[4])
                time = float(items[7])
                
                jacobians = np.array(list(map(lambda x: Interval(x).get_abs_max(), arr.split(';')[:-1]))).reshape(NUM_IMAGES, 20)
                
                avg_norm = 0
                for jacobi in jacobians:
                    jacobi = jacobi.reshape(2, 10).T
                    avg_norm += la.norm(jacobi, np.inf)
                avg_norm /= NUM_IMAGES
                
                add = 0
                for isize in interval_sizes:
                    if np.isclose(isize, interval_size):
                        add = 1
                        break
                if add:
                    data.append((interval_size, time, avg_norm))
                    
            all_data_sound[dataset][net][perturbation] = data            

In [8]:
from statistics import median as med

table = r"""
\begin{tabular}{c|c|c|c|c|c|c}
& Haze & Contrast & Rotation & Haze-Rotation & Contrast-Rotation & Contrast-Haze  \\ 
\hline
\multirow{2}{*}{MNIST ConvBig} & 0 & 0 & 0 & 0 & 0 & 0 \\
                               & 0 & 0 & 0 & 0 & 0 & 0 \\
\hline
\multirow{2}{*}{CIFAR ConvMed} & 0 & 0 & 0 & 0 & 0 & 0 \\
                               & 0 & 0 & 0 & 0 & 0 & 0 \\
\hline
\multirow{2}{*}{CIFAR ConvBig} & 0 & 0 & 0 & 0 & 0 & 0 \\
                               & 0 & 0 & 0 & 0 & 0 & 0
\end{tabular}"""
table = table.replace('0', 'holder')

perturbations = ['Haze', 'ContrastVariation', 'Rotation', 'HazeThenRotation', 'ContrastVariationThenRotation', 'ContrastVariationThenHaze']

for dataset, net in [('MNIST','ConvBig'), ('CIFAR','ConvMed'), ('CIFAR','ConvBig')]:
    # relative error
    for perturbation in perturbations:
        errors_norm = []
        time_overhead = []
        for orig, sound in zip(all_data[dataset][net][perturbation], all_data_sound[dataset][net][perturbation]):
            errors_norm.append(abs(orig[2] - sound[2]) / abs(orig[2]))
            time_overhead.append(round(sound[1] / orig[1], 3))
        table = table.replace('holder', f'{max(errors_norm):.2e}', 1)
    
    # time overhead
    for perturbation in perturbations:
        errors_norm = []
        time_overhead = []
        for orig, sound in zip(all_data[dataset][net][perturbation], all_data_sound[dataset][net][perturbation]):
            errors_norm.append(abs(orig[2] - sound[2]) / abs(orig[2]))
            time_overhead.append(round(sound[1] / orig[1], 3))
        table = table.replace('holder', f'{round(max(time_overhead), 2)}', 1)

print(table)


\begin{tabular}{c|c|c|c|c|c|c}
& Haze & Contrast & Rotation & Haze-Rotation & Contrast-Rotation & Contrast-Haze  \\ 
\hline
\multirow{2}{*}{MNIST ConvBig} & 2.24e-11 & 9.34e-13 & 1.23e-12 & 1.24e-12 & 1.25e-12 & 9.77e-13 \\
                               & 3.67 & 3.78 & 3.51 & 3.59 & 3.58 & 4.08 \\
\hline
\multirow{2}{*}{CIFAR ConvMed} & 8.08e-12 & 1.79e-12 & 8.93e-13 & 9.07e-13 & 8.99e-13 & 5.62e-12 \\
                               & 3.58 & 3.68 & 2.83 & 3.28 & 3.25 & 3.45 \\
\hline
\multirow{2}{*}{CIFAR ConvBig} & 2.84e-11 & 2.81e-12 & 1.30e-12 & 1.36e-12 & 1.36e-12 & 6.85e-12 \\
                               & 3.45 & 3.43 & 2.94 & 3.31 & 3.27 & 3.66
\end{tabular}
